In [ ]:
dbutils.widgets.text('src_catalog', '', '01 Source Catalog')
dbutils.widgets.text('src_schema', '', '02 Source Schema')
dbutils.widgets.text('src_table', '', '03 Source Table')
dbutils.widgets.text('batch_id', '', '04 Batch ID')
dbutils.widgets.text('tgt_catalog', '', '05 Target Catalog')
dbutils.widgets.text('tgt_schema', '', '06 Target Schema')
dbutils.widgets.text('tgt_table', '', '07 Target Table')

In [ ]:
from lakefed_ingest.main import *

In [ ]:
src_catalog = dbutils.widgets.get('src_catalog')
src_schema = dbutils.widgets.get('src_schema')
src_table = dbutils.widgets.get('src_table')
batch_id = int(dbutils.widgets.get('batch_id'))
tgt_catalog = dbutils.widgets.get('tgt_catalog')
tgt_schema = dbutils.widgets.get('tgt_schema')
tgt_table = dbutils.widgets.get('tgt_table')

In [ ]:
print(f'Batch id: {batch_id}')

# TODO set partitions table suffix at job level
partitions_tbl = f'{tgt_catalog}.{tgt_schema}.{tgt_table}_partitions'

batch_of_partitions_qry = f"""\
    select array_agg(where_clause)
    from {partitions_tbl}
    where batch_id = {batch_id}
"""

batch_of_partitions = spark.sql(batch_of_partitions_qry).collect()[0][0]

print(f'Batch size: {len(batch_of_partitions)}')

# Assign batch list to job task value. The list can be iterated in a subsequent task.
dbutils.jobs.taskValues.set(key="batch_of_partitions", value=batch_of_partitions)